In [1]:
import time
from uuid import uuid4

from dotenv import load_dotenv
import pandas as pd
import promptquality as pq
from promptquality import NodeType, NodeRow


load_dotenv("../.env")
pq.login('https://console.demo.rungalileo.io')

👋 You have logged into 🔭 Galileo (https://console.demo.rungalileo.io/) as pratik@rungalileo.io.


Config(console_url=Url('https://console.demo.rungalileo.io/'), username='pratik@rungalileo.io', password=SecretStr('**********'), api_key=None, token=SecretStr('**********'), current_user='pratik@rungalileo.io', current_project_id=None, current_project_name=None, current_run_id=None, current_run_name=None, current_run_url=None, current_run_task_type=None, current_template_id=None, current_template_name=None, current_template_version_id=None, current_template_version=None, current_template=None, current_dataset_id=None, current_job_id=None, current_prompt_optimization_job_id=None, api_url=Url('https://api.demo.rungalileo.io/'))

In [2]:
project_name = "rag-syn-data-cleaning"
run_name="gpt-4o-2024-08-06-v1"

config = {
    "groundedness": True,
    "toxicity": False,
    "factuality": False,
    "context_relevance": False,
    "sexist": False,
    "pii": False,
    "prompt_perplexity": False,
    "chunk_attribution_utilization_gpt": False,
    "completeness_gpt": False,
    "tone": False,
    "prompt_injection": False
  }

df = pd.read_parquet("../data/syn_data_rag/output/test.parquet").sample(1000, random_state=0)
df.head()

,sector,company,context,question_type,question,answer,prompt
1830,Consumer Staples,J.M. Smucker Company (The),"\n1. During the quarter, Smucker’s investments...",tabular,How much more did Smucker spend on marketing f...,Smucker spent $120 million on marketing for th...,"Context: \n1. During the quarter, Smucker’s in..."
994,Industrials,CH Robinson,\n1. The company's focus on sustainability ini...,math,By how much did CH Robinson reduce its carbon ...,CH Robinson reduced its carbon emissions by 15...,Context: \n1. The company's focus on sustainab...
6290,Technology,Tech Innovations Inc.,"\n1. Looking forward, Tech Innovations Inc. ai...",cannot answer,What was Tech Innovations Inc.'s revenue from ...,I do not have the information to answer this q...,"Context: \n1. Looking forward, Tech Innovation..."
3848,Consumer Staples,Keurig Dr Pepper,\n1. A new marketing initiative saw Keurig Dr ...,extractive,Which new product did PepsiCo introduce that b...,Pepsi Cafe is the new product introduced by Pe...,Context: \n1. A new marketing initiative saw K...
3296,Information Technology,Tech Innovators Inc.,"\n1. Looking ahead, Tech Innovators is optimis...",tabular,What was the percentage increase in Tech Innov...,The percentage increase in Market Share from Q...,"Context: \n1. Looking ahead, Tech Innovators i..."


In [3]:
print(df.prompt.values[0])

Context: 
1. During the quarter, Smucker’s investments in e-commerce have paid off, with online sales growing by 25%. The company is enhancing its digital infrastructure to provide a seamless purchasing experience. Competitors such as Nestlé have also been strengthening their e-commerce capabilities, signaling a shift in consumer buying behavior towards online platforms.
2. In terms of financial health, the company has reduced its operating expenses by 3% compared to the previous year. This reduction is part of an overall cost optimization initiative aimed at increasing profitability. General administrative expenses remain unchanged, while marketing and R&D expenses have seen slight increases to support brand innovation and growth.
3. With the rise in consumer demand for transparency in food sourcing, Smucker has been actively engaging in sustainable sourcing initiatives. This move not only aligns with consumer expectations but also strengthens the company's brand equity. Competitors l

In [4]:
# create nodes
nodes = []
for prompt, response in zip(df.prompt, df.answer):
    root_id = uuid4()
    llm_id = uuid4()
    
    nodes.append(NodeRow(
                node_id=root_id, 
                chain_root_id=root_id, 
                node_type=NodeType.chain, 
                step=0, 
                node_input=prompt, 
                node_output=response,
            ))
    nodes.append(NodeRow(
                node_id=llm_id, 
                chain_root_id=root_id, 
                chain_id=root_id,
                node_type=NodeType.llm, 
                step=1, 
                node_input=prompt, 
                node_output=response,
            ))

# push to galileo
pq.chain_run(
    rows=nodes,
    scorers_config=pq.ScorersConfiguration(**config),
    project_name=project_name,
    run_name=run_name
)
time.sleep(30)

Processing chain run...:   0%|          | 0/5 [00:00<?, ?it/s]

Initial job complete, executing scorers asynchronously. Current status:
cost: Computing 🚧
protect_status: Done ✅
latency: Done ✅
groundedness: Computing 🚧
🔭 View your prompt run on the Galileo console at: https://console.demo.rungalileo.io/prompt/chains/cc3568f4-ad31-4109-b5ba-f075276775a5/a2d786ce-3891-4336-9134-7eac69770ea7?taskType=12
